# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
cities_weather_df = pd.read_csv("output_data/cities.csv")
cities_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8061,-68.3185,35.29,80,40,1.14,AR,1659622809
1,Asunción,-25.2800,-57.6344,68.14,77,75,12.66,PY,1659622810
2,Broome,-17.9567,122.2240,64.29,88,9,3.44,AU,1659622811
3,Georgetown,6.8137,-58.1624,79.09,100,75,2.30,GY,1659622811
4,Bredasdorp,-34.5331,20.0428,65.79,52,65,4.27,ZA,1659622812
...,...,...,...,...,...,...,...,...,...
547,Minna,9.6187,6.5476,82.69,69,40,8.05,NG,1659623148
548,Sanandij,35.3126,46.9979,98.85,7,0,9.22,IR,1659623148
549,Guayabal,5.0323,-74.8832,79.63,76,100,1.61,CO,1659623149
550,Ust'-Kamchatsk,56.2400,162.5370,56.57,83,62,6.22,RU,1659623150


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
gmaps.configure(api_key=g_key)
locations = cities_weather_df[["Lat", "Lng"]]
humidity = cities_weather_df['Humidity']

In [22]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2.5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
ideal_cities_df = cities_weather_df.loc[(cities_weather_df['Max Temp'] <= 80) & (cities_weather_df['Max Temp'] >= 70)
                                       &(cities_weather_df['Cloudiness'] == 0)]
ideal_cities_df = ideal_cities_df.dropna()
ideal_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,Honiton,50.7852,-3.1883,72.91,49,0,11.50,GB,1659622856
96,Wuwei,37.9289,102.6360,76.26,63,0,5.19,CN,1659622869
98,Hilo,19.7074,-155.0816,76.19,82,0,4.61,US,1659622870
128,Arraial do Cabo,-22.9663,-42.0244,76.98,73,0,16.11,BR,1659622889
153,Muquiçaba,-20.6719,-40.4985,78.93,54,0,10.89,BR,1659622905
265,Makakilo,21.3469,-158.0858,72.99,78,0,4.61,US,1659622974
317,Cortez,37.3495,-108.5841,72.68,63,0,3.44,US,1659623006
334,Alpena,45.0617,-83.4328,74.17,75,0,6.91,US,1659623016
429,Kahului,20.8894,-156.4727,76.78,68,0,3.00,US,1659623075
434,Makaha,21.4729,-158.2107,74.32,78,0,4.92,US,1659623078


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df = ideal_cities_df
hotel_df['Hotel Name']=""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
75,Honiton,50.7852,-3.1883,72.91,49,0,11.50,GB,1659622856,
96,Wuwei,37.9289,102.6360,76.26,63,0,5.19,CN,1659622869,
98,Hilo,19.7074,-155.0816,76.19,82,0,4.61,US,1659622870,
128,Arraial do Cabo,-22.9663,-42.0244,76.98,73,0,16.11,BR,1659622889,
153,Muquiçaba,-20.6719,-40.4985,78.93,54,0,10.89,BR,1659622905,
265,Makakilo,21.3469,-158.0858,72.99,78,0,4.61,US,1659622974,
317,Cortez,37.3495,-108.5841,72.68,63,0,3.44,US,1659623006,
334,Alpena,45.0617,-83.4328,74.17,75,0,6.91,US,1659623016,
429,Kahului,20.8894,-156.4727,76.78,68,0,3.00,US,1659623075,
434,Makaha,21.4729,-158.2107,74.32,78,0,4.92,US,1659623078,


In [28]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    
    (lat,lng) = hotel_df.loc[index, ['Lat','Lng']]

    params = {
        "location": f"{lat},{lng}", 
        "radius": "5000",
        "type": "hotel",
        "key": g_key,
    }

    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    #pprint(results)
    #input("Continue?")
    try:
        print(f"Closest hotel is {results[1]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[1]['name']
     
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Closest hotel is The Heathfield Inn.
------------
Closest hotel is Haizang Park.
------------
Closest hotel is SCP Hilo Hotel.
------------
Closest hotel is Pousada Porto Praia.
------------
Closest hotel is Hotel Porto do Sol.
------------
Closest hotel is Wet 'n' Wild Hawaii.
------------
Closest hotel is Sand Canyon National 9 Inn.
------------
Closest hotel is Days Inn by Wyndham Alpena.
------------
Closest hotel is Maui Beach Hotel.
------------
Closest hotel is Mākaha Beach Park.
------------
Closest hotel is Kennedy Energy Park.
------------
Closest hotel is Park Alandia Hotel.
------------
Closest hotel is Courtyard by Marriott Lincoln Downtown/Haymarket.
------------
Closest hotel is Maxi Super.
------------
Closest hotel is Via Field.
------------
Closest hotel is Norfolk Southern.
------------


In [29]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
75,Honiton,50.7852,-3.1883,72.91,49,0,11.50,GB,1659622856,The Heathfield Inn
96,Wuwei,37.9289,102.6360,76.26,63,0,5.19,CN,1659622869,Haizang Park
98,Hilo,19.7074,-155.0816,76.19,82,0,4.61,US,1659622870,SCP Hilo Hotel
128,Arraial do Cabo,-22.9663,-42.0244,76.98,73,0,16.11,BR,1659622889,Pousada Porto Praia
153,Muquiçaba,-20.6719,-40.4985,78.93,54,0,10.89,BR,1659622905,Hotel Porto do Sol
265,Makakilo,21.3469,-158.0858,72.99,78,0,4.61,US,1659622974,Wet 'n' Wild Hawaii
317,Cortez,37.3495,-108.5841,72.68,63,0,3.44,US,1659623006,Sand Canyon National 9 Inn
334,Alpena,45.0617,-83.4328,74.17,75,0,6.91,US,1659623016,Days Inn by Wyndham Alpena
429,Kahului,20.8894,-156.4727,76.78,68,0,3.00,US,1659623075,Maui Beach Hotel
434,Makaha,21.4729,-158.2107,74.32,78,0,4.92,US,1659623078,Mākaha Beach Park


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))